In [2]:
import os
import json
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import import_ipynb

# Load your model architecture
from CNN_proposals_2 import ProposalClassifierCNN, ProposalClassifierResNet, ProposalClassifierVGG16   # Replace with your model class

# Load the trained model
#model = ProposalClassifierVGG16(2)
#model = ProposalClassifierResNet(2)
model = ProposalClassifierCNN(2)
model.load_state_dict(torch.load('/home/arth/Desktop/various/CV/poster3/proposal_classifier_NMS_SS.pth'))
model.eval()

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/tmp/ipykernel_208357/2579793710.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/arth/Desktop/various/CV/poster3/proposal_classi

ProposalClassifierCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

Load the test images

In [3]:
# Load the splits.json
with open('/home/arth/Desktop/various/CV/poster3/Potholes/Potholes/splits.json', 'r') as f:
    splits = json.load(f)

test_images_xml = splits['test']  # List of test image XML filenames

# Convert XML filenames to image filenames
test_image_filenames = [filename.replace('.xml', '.jpg') for filename in test_images_xml]

# Define directories
image_dir = '/home/arth/Desktop/various/CV/poster3/Potholes/Potholes/annotated-images'      # Replace with your image directory
proposal_dir = '/home/arth/Desktop/various/CV/poster3/Potholes/Potholes/NMS/Labeled_Proposals_SS'  # Replace with your proposals directory


Image transformatiions

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.Pad((0, 0, 256, 256), fill=0, padding_mode='constant'),  # Adjust padding as needed
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])


In [5]:
import xml.etree.ElementTree as ET
import logging

def parse_voc_xml(xml_file):
    """
    Parse PASCAL VOC XML file to extract ground truth bounding boxes.
    """
    try:
        tree = ET.parse(xml_file)
    except ET.ParseError as e:
        logging.error(f"Error parsing XML file {xml_file}: {e}")
        return []
    except FileNotFoundError:
        logging.error(f"XML file not found: {xml_file}")
        return []

    root = tree.getroot()
    boxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        if bbox is None:
            continue
        try:
            x1 = int(float(bbox.find('xmin').text))
            y1 = int(float(bbox.find('ymin').text))
            x2 = int(float(bbox.find('xmax').text))
            y2 = int(float(bbox.find('ymax').text))
            if x1 >= x2 or y1 >= y2:
                logging.warning(f"Invalid bounding box in {xml_file}: {bbox.text}")
                continue
            boxes.append([x1, y1, x2, y2])
        except (AttributeError, ValueError) as e:
            logging.warning(f"Malformed bounding box in {xml_file}: {e}")
            continue
    return boxes


Process each test image

In [6]:
from tqdm import tqdm  # For progress bar

all_detections = []
all_annotations = []

for img_filename in tqdm(test_image_filenames):
    # Load the image
    img_path = os.path.join(image_dir, img_filename)
    image = Image.open(img_path).convert('RGB')
    image_np = np.array(image)

    # Load the proposals
    proposal_filename = img_filename.replace('.jpg', '_labeled_proposals.npy')
    proposal_path = os.path.join(proposal_dir, proposal_filename)
    proposals = np.load(proposal_path, allow_pickle=True)  # Shape: (num_proposals, 4)

    detections = []

    for proposal in proposals:
    # Ensure proposal is a dictionary
        if not isinstance(proposal, dict):
            logging.warning(f"Unexpected proposal format in {proposal_path}: {proposal}")
            continue

        # Extract 'box' and 'label'
        try:
            box = proposal['box']
            label = proposal['label']
        except KeyError as e:
            logging.warning(f"Missing key {e} in proposal: {proposal} in file {proposal_path}")
            continue

        # Optionally, filter out background proposals
        if label.lower() == 'background':
            continue  # Skip background proposals

        # Extract coordinates from the 'box' array
        if len(box) != 4:
            logging.warning(f"Invalid box format in proposal: {proposal} in file {proposal_path}")
            continue

        x, y, w, h = box
        try:
            x1 = int(x)
            y1 = int(y)
            x2 = int(x + w)
            y2 = int(y + h)
        except (TypeError, ValueError) as e:
            logging.warning(f"Invalid box values in proposal: {proposal} in file {proposal_path}")
            continue

        # Validate bounding box coordinates
        if x1 >= x2 or y1 >= y2:
            logging.warning(f"Invalid bounding box coordinates: {proposal} in file {proposal_path}")
            continue

        # Proceed with cropping and processing
        try:
            cropped_region = image.crop((x1, y1, x2, y2))
        except Exception as e:
            logging.warning(f"Error cropping image {img_filename} with bbox {proposal}: {e}")
            continue

        input_tensor = preprocess(cropped_region).unsqueeze(0).to(device)

        # Inference
        with torch.no_grad():
            output = model(input_tensor)
            # Assuming the model outputs logits; adjust if necessary
            if output.dim() == 1:
                output = output.unsqueeze(0)
            scores = torch.softmax(output, dim=1)
            score, predicted_class = torch.max(scores, dim=1)
            score = score.item()
            predicted_class = predicted_class.item()

        detections.append({
            'bbox': [x1, y1, x2, y2],
            'score': score,
            'class': predicted_class
        })

    # Apply NMS (to be implemented in the next step)
    # Store detections and annotations for evaluation
    all_detections.append({'image_id': img_filename, 'detections': detections})

    # Load ground truth annotations
    xml_filename = img_filename.replace('.jpg', '.xml')
    xml_path = os.path.join('/home/arth/Desktop/various/CV/poster3/Potholes/Potholes/annotated-images', xml_filename)
    gt_boxes = parse_voc_xml(xml_path)  # Function to parse XML files (defined later)
    all_annotations.append({'image_id': img_filename, 'bboxes': gt_boxes})


100%|██████████| 133/133 [00:01<00:00, 84.10it/s] 


In [7]:
import numpy as np

# Load a single proposal file
proposal_path = '/home/arth/Desktop/various/CV/poster3/Potholes/Potholes/NMS/Labeled_Proposals_SS/img-529_labeled_proposals.npy'  # Replace with an actual file path
proposals = np.load(proposal_path, allow_pickle=True)

# Check the type of the first proposal
first_proposal = proposals[0]
print(type(first_proposal))
print(first_proposal)

<class 'dict'>
{'box': array([289, 270,  45,  30]), 'label': 'background'}


NMS

In [8]:
def nms(boxes, scores, iou_threshold=0.5):
    import numpy as np

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= iou_threshold)[0]
        order = order[inds + 1]
    return keep


Apply the NMS to the Detections


In [9]:
# After collecting all detections
boxes = np.array([d['bbox'] for d in detections])
scores = np.array([d['score'] for d in detections])

if len(boxes) > 0:
    keep_indices = nms(boxes, scores, iou_threshold=0.5)
    final_detections = [detections[i] for i in keep_indices]
else:
    final_detections = []

# Update all_detections with NMS results
all_detections[-1]['detections'] = final_detections


Evaluate Object Detection with Average Precision(AP)

In [10]:
import xml.etree.ElementTree as ET

def parse_voc_xml(xml_file):
    tree=ET.åarse(xml_file)
    root=tree.groot()
    boxes=[]
    for obj in root.findall('object'):
        bbox= obj.find('bndbox')
        x1=int(float(bbox.find('xmin').text))
        y1=int(float(bbox.find('ymin').text))
        x2=int(float(bbox.find('xmax').text))
        y2=int(float(bbox.find('xymax').text))
        boxes.append([x1, y1, x2, y2])
    return boxes

Copmute Intersections over Union(IoU)

In [11]:
def compute_iou(box1, box2):
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])

    inter_area = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)
    if inter_area == 0:
        return 0.0

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area


Evaluate the Detections and compute AP

In [20]:
def evaluate_detections(all_detections, all_annotations, iou_threshold=0.8):
    all_tp = []
    all_fp = []
    all_scores = []
    num_gt_boxes = 0

    # Create a mapping from image_id to ground truth boxes
    gt_dict = {ann['image_id']: ann['bboxes'] for ann in all_annotations}

    for image_info in all_detections:
        image_id = image_info['image_id']
        detections = image_info['detections']
        gt_boxes = gt_dict.get(image_id, [])
        num_gt_boxes += len(gt_boxes)
        detected = [False] * len(gt_boxes)

        for det in detections:
            best_iou = 0.0
            best_gt_idx = -1
            for idx, gt_box in enumerate(gt_boxes):
                iou = compute_iou(det['bbox'], gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            if best_iou >= iou_threshold and not detected[best_gt_idx]:
                all_tp.append(1)
                all_fp.append(0)
                detected[best_gt_idx] = True
            else:
                all_tp.append(0)
                all_fp.append(1)
            all_scores.append(det['score'])

    # Convert to numpy arrays
    all_tp = np.array(all_tp)
    all_fp = np.array(all_fp)
    all_scores = np.array(all_scores)

    # Sort by scores
    sorted_indices = np.argsort(-all_scores)
    all_tp = all_tp[sorted_indices]
    all_fp = all_fp[sorted_indices]

    # Cumulative sums
    cum_tp = np.cumsum(all_tp)
    cum_fp = np.cumsum(all_fp)

    recalls = cum_tp / num_gt_boxes
    precisions = cum_tp / (cum_tp + cum_fp)

    # Compute AP using the trapezoidal rule
    ap = np.trapz(precisions, recalls)

    return ap, precisions, recalls


Calculate AP

In [21]:
ap, precisions, recalls = evaluate_detections(all_detections, all_annotations)
print(f'Average Precision (AP): {ap:.4f}')


Average Precision (AP): 0.0166


/tmp/ipykernel_208357/3117772641.py:52: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  ap = np.trapz(precisions, recalls)
